# Silver Layer - Clean and Structure the Data

🎯 **Goal**: Make the data clean, consistent, and usable for downstream tasks

🔧 **Tasks a data engineer performs**:
- Read from the bronze layer
- Flatten nested structures (e.g. explode arrays)
- Drop invalid or duplicate rows
- Convert data types to the correct formats (e.g., height as float)
- Rename columns for clarity or consistency
- Store result as a new dataset

### 🧹 This notebook: Silver Layer – Cleaned & Structured Pokémon Data

In this notebook, we:
- Read the raw Pokémon data from the bronze Delta table
- Extract selected fields from the nested `raw_json` column
- Explode arrays such as `types` into individual rows
- Ensure data types are consistent (e.g. height as float)
- Rename or restructure columns for clarity
- Save the cleaned data to a new Delta table in the silver layer at `../data/silver/pokemon`

This layer prepares the data for analysis and ML by enforcing structure and consistency.


# Step 1: Import Spark and setup the session


In [27]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, explode

spark = SparkSession.builder \
    .appName("DemoPipe - Silver Layer") \
    .config("spark.jars.packages", "io.delta:delta-spark_2.12:3.3.0") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .getOrCreate()

# Confirm Spark is running
spark.version

'3.5.5'

# Step 2: Define paths to bronze and silver tables

In [28]:
bronze_path = "../data/bronze/pokemon"
silver_path = "../data/silver/pokemon"

# For Databricks:
# bronze_path = "dbfs:/tmp/bronze/pokemon"
# silver_path = "dbfs:/tmp/silver/pokemon"

# For Microsoft Fabric:
# bronze_path = "Tables/bronze_pokemon"
# silver_path = "Tables/silver_pokemon"

# Step 3: Load the raw bronze Delta table

In [29]:
bronze_df = spark.read.format("delta").load(bronze_path)
bronze_df.printSchema()
bronze_df.show(1, truncate=False)

root
 |-- id: long (nullable = true)
 |-- name: string (nullable = true)
 |-- raw_json: map (nullable = true)
 |    |-- key: string
 |    |-- value: array (valueContainsNull = true)
 |    |    |-- element: map (containsNull = true)
 |    |    |    |-- key: string
 |    |    |    |-- value: map (valueContainsNull = true)
 |    |    |    |    |-- key: string
 |    |    |    |    |-- value: string (valueContainsNull = true)

+---+------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

# Step 4: Extract structured fields from raw JSON

In [31]:
df = bronze_df.select(
    col("id"),
    col("name"),
    col("raw_json.types").alias("types"),
    col("raw_json.height").cast("int").alias("height"),
    col("raw_json.weight").cast("int").alias("weight"),
    col("raw_json.base_experience").cast("int").alias("base_experience")
)

AnalysisException: [DATATYPE_MISMATCH.CAST_WITHOUT_SUGGESTION] Cannot resolve "CAST(raw_json[height] AS INT)" due to data type mismatch: cannot cast "ARRAY<MAP<STRING, MAP<STRING, STRING>>>" to "INT".;
'Project [id#1395L, name#1396, raw_json#1397[types] AS types#1496, cast(raw_json#1397[height] as int) AS height#1497, cast(raw_json#1397[weight] as int) AS weight#1498, cast(raw_json#1397[base_experience] as int) AS base_experience#1499]
+- Relation [id#1395L,name#1396,raw_json#1397] parquet


In [30]:
# Extract from the raw_json map:
df = bronze_df.select(
    col("id"),
    col("name"),
    col("raw_json.height"),
    col("raw_json.weight"),
    col("raw_json.base_experience"),
    col("raw_json.types.type_1").alias("type_1"),
    col("raw_json.types.type_2").alias("type_2"),
    col("raw_json.abilities.ability_1").alias("ability_1"),
    col("raw_json.abilities.ability_2").alias("ability_2"),
    col("raw_json.abilities.hidden_ability").alias("hidden_ability")
)
# Flatten the types and abilities columns  
df = df.withColumn("type_1", explode(col("type_1"))) \
    .withColumn("type_2", explode(col("type_2"))) \
    .withColumn("ability_1", explode(col("ability_1"))) \
    .withColumn("ability_2", explode(col("ability_2"))) \
    .withColumn("hidden_ability", explode(col("hidden_ability")))

# Remove null values
df = df.na.drop()   
# Remove duplicates
df = df.dropDuplicates(["id", "name", "type_1", "type_2", "ability_1", "ability_2", "hidden_ability"])

df.head(5)

AnalysisException: [DATATYPE_MISMATCH.UNEXPECTED_INPUT_TYPE] Cannot resolve "raw_json[types][type_1]" due to data type mismatch: Parameter 2 requires the "INTEGRAL" type, however "type_1" has the type "STRING".;
'Project [id#1395L, name#1396, raw_json#1397[height] AS height#1488, raw_json#1397[weight] AS weight#1489, raw_json#1397[base_experience] AS base_experience#1490, raw_json#1397[types][type_1] AS type_1#1483, raw_json#1397[types][type_2] AS type_2#1484, raw_json#1397[abilities][ability_1] AS ability_1#1485, raw_json#1397[abilities][ability_2] AS ability_2#1486, raw_json#1397[abilities][hidden_ability] AS hidden_ability#1487]
+- Relation [id#1395L,name#1396,raw_json#1397] parquet


In [ ]:
# Write to the silver layer
df.write.format("delta").mode("overwrite").save(silver_path)
# Create a Delta table
spark.sql(f"""
    CREATE TABLE IF NOT EXISTS silver_pokemon
    USING DELTA
    LOCATION '{silver_path}'
""")
# Create a view for the silver table 
spark.sql(f"""
    CREATE OR REPLACE TEMP VIEW silver_pokemon AS
    SELECT * FROM silver_pokemon
""")
# Check the silver table
silver_df = spark.sql("SELECT * FROM silver_pokemon")
silver_df.printSchema()
silver_df.show(5)